In [0]:
@logs
def forecasted_data(df):
    from datetime import datetime,timedelta
    from pyspark.sql.functions import hour,col,lit,min,max,round,mean
    df=df.withColumn('time_id',hour('created_on')).withColumn('date',col('created_on').cast('date'))
    time_id,date=df.select('time_id','date').first()
    date_id=str(date).replace('-','')
    try:
        query=f"Delete from fact_hourly_weather where time_id>='{time_id}' and date_id>='{date_id}';"
        spark.sql(query)
    except:
        pass
    date_df=spark.table('dim_date')
    time_df=spark.table('dim_time')
    
    joined_df = df.join(date_df, df['date'] == date_df['fullDate'], 'inner')\
    .join(time_df, df['time_id'] == time_df['time_id'], 'inner')\
    .select(date_df['Date_id'], df['date'], df['dt'], df['time_id'], df['city_id'], df['temp'], df['timezone'], df['temp_min'], df['temp_max'], df['pressure'], df['humidity'], df['visibility'])
    
    fact_df=joined_df.withColumn("IsForecasted",lit('false'))\
            .select('time_id','date_id','city_id','temp','temp_min','temp_max','pressure','humidity','visibility','IsForecasted')
    
    for i in range(1,4):
        ref_time=(time_id-i)%24
        off_date=int((time_id-i)//24)
        ref_date=date+timedelta(off_date)
        ref_date=str(ref_date).replace('-','')
        
        try:
            query=f"select * from fact_hourly_weather where time_id='{ref_time}' and date_id='{ref_date}'"\
                    .drop('load_run_id','created_on','created_by')
            df1=spark.sql(query)
            fact_df=fact_df.union(df1)
        except:
            pass
        
    for i in range(1,4):
        ref_time=(time_id+i)%24
        off_date=int((time_id+i)//24)
        ref_date=date+timedelta(off_date)
        ref_date=str(ref_date).replace('-','')

        forecasted_df=fact_df.groupBy(col('city_id'),col('date_id'))\
                    .agg(round(mean('temp'),2).alias('temp'),
                         min('temp_min').alias('temp_min'),
                         max('temp_max').alias('temp_max'),
                         mean('pressure').cast('int').alias('pressure'),
                         mean('humidity').cast('int').alias('humidity'),
                         mean('visibility').cast('int').alias('visibility'), 
                       )
        forecasted_df=forecasted_df.withColumn('time_id',lit(ref_time)).withColumn('date_id',lit(ref_date)).withColumn('IsForecasted',lit('true'))\
                        .select('time_id', 'date_id', 'city_id', 'temp', 'temp_min', 'temp_max', 'pressure', 'humidity', 'visibility','IsForecasted')
        fact_df=fact_df.union(forecasted_df)
        not_ref=(ref_time-4)%24
        fact_df=fact_df.filter(fact_df.time_id!=not_ref)
            
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return fact_df,start,end
        

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3089831312894099>:1
----> 1 @logs
      2 def forecasted_data(df):
      3     from datetime import datetime,timedelta
      4     from pyspark.sql.functions import hour,col,lit,min,max,round,mean

NameError: name 'logs' is not defined